# Geospatial API Project
Silver Sloths - Habeeb Branch<br>
October 2, 2021

In [1]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
from io import StringIO
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import re

In [2]:
# Read in all aggravated burglary data between January 1, 2021 and June 30, 2021
burglary_endpoint =  "https://data.nashville.gov/resource/2u6v-ujjs.geojson"
params = {
    "$where": "incident_reported between '2021-01-01T00:00:00' and '2021-06-30T23:59:59'",
    'offense_description': 'BURGLARY- AGGRAVATED',
    '$limit': '1000000'
}
burglary_response = requests.get(burglary_endpoint,  
                                 params = params)
burglary = gpd.read_file(StringIO(burglary_response.text))

In [3]:
# Drop duplicate incidents
burglary = burglary.drop_duplicates(subset='incident_number')

# Read in census tract shape files
census_shape = gpd.read_file('../data/tl_2019_47_tract.shp')
burglary.crs = census_shape.crs

# Spatial Join between Burglary and Census Tract Shape Data
burglary_in_census_tract = gpd.sjoin(burglary, census_shape, op='within')

# Rename columns and drop unneeded columns
burglary_in_census_tract = (burglary_in_census_tract.rename({'NAMELSAD':'CENSUS_TRACT',
                                                           'TRACTCE': 'CENSUS_BLOCK'}, 
                                                           axis='columns')
                            .drop(columns=['index_right','GEOID','NAME','MTFCC','FUNCSTAT','ALAND','AWATER', 'index_right']))

# Count up the amount of assaulted burglaries in each census tract
burglary_in_census_tract['CENSUS_TRACT'].value_counts()

Census Tract 163       53
Census Tract 162       21
Census Tract 110.01    20
Census Tract 195       19
Census Tract 193       19
                       ..
Census Tract 179.02     1
Census Tract 103.03     1
Census Tract 178        1
Census Tract 132.02     1
Census Tract 110.02     1
Name: CENSUS_TRACT, Length: 148, dtype: int64

Census tract 163 has the most aggravated burglary cases between Janury 1, 2021 and June 30, 2021 at 53 cases.

In [4]:
# Read in API key for census site
with open('../data/census.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['api_key']

# Read in population data from Census website
population_endpoint = "https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key={}".format(api_key)
population_response = requests.get(population_endpoint)
population = pd.read_csv(StringIO(population_response.text))

# Read in median income data from Census website
median_income_endpoint = "https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,S1901_C01_012E&for=tract:*&in=state:47&in=county:037&key={}".format(api_key)
median_income_response = requests.get(median_income_endpoint)
median_income = pd.read_csv(StringIO(median_income_response.text))

In [5]:
# Extract and clean info from population index
census_tract = []
for length in range(len(population.index)):
    global census_tract
    census_tract.append(re.sub(r"\W{2}","",population.index[length][0]))
    
# Create a new column with cleaned data from above
population["CENSUS_TRACT"] = census_tract

# Reset index of dataframe
population.index = range(len(population.index))

In [6]:
# Create a new column with cleaned data from above
median_income["CENSUS_TRACT"] = census_tract

# Reset index of dataframe
median_income.index = range(len(median_income.index))

In [7]:
# Rename and drop columns in population
population = (population.drop(columns=['[["NAME"','state','county','Unnamed: 5', 'tract]'])
                 .rename({'B01001_001E':'population'},
                         axis='columns'))

# Rename and drop columns in median_income
median_income = (median_income.drop(columns=['[["NAME"','state','county','Unnamed: 5', 'tract]'])
                 .rename({'S1901_C01_012E':'median_income'},
                         axis='columns'))

In [8]:
burglary_income_population = pd.merge(burglary_in_census_tract,population,on="CENSUS_TRACT")
burglary_income_population = pd.merge(burglary_income_population,median_income,on="CENSUS_TRACT")

In [9]:
burglary_income_population

,victim_county_resident,zip_code,victim_number,offense_nibrs,rpa,latitude,victim_race,incident_number,investigation_status,offense_number,...,primary_key,geometry,STATEFP,COUNTYFP,CENSUS_BLOCK,CENSUS_TRACT,INTPTLAT,INTPTLON,population,median_income
0,RESIDENT,None,1,220,8203,36.15,B,20210249540,Open,1,...,20210249540_11,POINT (-86.77000 36.15000),47,037,016000,Census Tract 160,+36.1454955,-086.7638143,945,37083
1,RESIDENT,None,1,220,8009,36.14,W,20210187070,Open,1,...,20210187070_11,POINT (-86.76000 36.14000),47,037,016000,Census Tract 160,+36.1454955,-086.7638143,945,37083
2,RESIDENT,None,1,220,8011,36.14,A,20210006869,Open,1,...,20210006869_11,POINT (-86.76000 36.14000),47,037,016000,Census Tract 160,+36.1454955,-086.7638143,945,37083
3,RESIDENT,None,1,220,8153,36.15,W,20210223220,Open,1,...,20210223220_11,POINT (-86.77000 36.15000),47,037,016000,Census Tract 160,+36.1454955,-086.7638143,945,37083
4,RESIDENT,None,1,220,8137,36.14,B,20210013709,Open,1,...,20210013709_11,POINT (-86.76000 36.14000),47,037,016000,Census Tract 160,+36.1454955,-086.7638143,945,37083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,RESIDENT,37013,1,220,8997,36.082,W,20210316971,Closed,1,...,20210316971_11,POINT (-86.60300 36.08200),47,037,015619,Census Tract 156.19,+36.0872112,-086.5882734,5105,74335
822,RESIDENT,37076,1,220,9509,36.198,W,20210344666,Closed,1,...,20210344666_11,POINT (-86.60200 36.19800),47,037,015405,Census Tract 154.05,+36.2059302,-086.5922143,4882,62143
823,RESIDENT,37220,2,220,8615,36.051,W,20210083306,Closed,1,...,20210083306_12,POINT (-86.75800 36.05100),47,037,018801,Census Tract 188.01,+36.0609910,-086.7571310,5889,101027
824,RESIDENT,37209,2,220,4521,36.168,W,20210225465,Closed,1,...,20210225465_12,POINT (-86.82200 36.16800),47,037,013602,Census Tract 136.02,+36.1685854,-086.8283325,1818,-666666666
